# Patrick de Carvalho Tavares Rezende Ferreira - RA: 175480 - EFC2

Repositório: https://github.com/patrickctrf/IA353-EG453-Redes-Neurais/tree/master/EFC2_IA353_1s2019

## Questão 1

Inicialmente, executou-se 5 vezes o código sugerido inicial a fim de verificar o desempenho da proposta. Seu desempenho foi de:

* Loss: 0.0733; Acurácia: 0.9775.

Utilizando-se o método de tentativa e erro, foi criado um script que verificava o desempenho da rede para diferentes parâmetros alterados como dropout (0.1 a 0.6), número de camadas (1 a 4 intermediárias), épocas de treinamento (4 a 8) e número de neurônios por camada (256 a 512). O script executava esta mudança de parâmetros dentro de loops "for" para executar todas as combinações possíveis e tirava também a média das múltiplas execuções com mesmos parâmetros, a fim de se obter uma média de desempenho mais confiável. Os resultados desta varredura eram salvos ao final das execuções em um arquivo "listas.txt", permitindo ao usuários verificar qual a configuração obteve melhor desempenho.
Foram utilizadas 4 threads - para varredura de redes de 1 a 4 camadas - durante o treinamento, a fim de promover paralelismo e diminuir o tempo requerido, que chegava a dezenas de horas.

Para a proposta final deste modelo, os parâmetros que resultaram no melhor desempenho durante a varredura foram:

* Camadas: 2; Neurônios por camada: 512; Dropout: 0.2; Épocas: 8.

O desempenho médio obtido foi de:

* Loss: 0.0693; Acurácia: 0.9828.

Ambas as soluções consumiram um tempo de execução da ordem de poucos minutos e a diferença de desempenho foi cerca de 0,5% em ganho.

Os arquivos utilizados foram (no diretório q1):

Proposta Inicial: q1Inicial.py
Script de Varredura de parâmetros: q1.py
Proposta Final: q1Final.py

In [ ]:
# q1Inicial.py

import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation=tf.nn.relu),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)
model_json = model.to_json()
json_file = open("model_MLP.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_MLP.h5")
print("Model saved to disk")
os.getcwd()

In [ ]:
# q1.py

import tensorflow as tf
import os
import threading

myMutex = threading.Lock()
value = "teste"

numeroDeNeuronios = []
numeroDeEpocas = []
numeroDeCamadas = []
numeroDeDropout = []
taxaDeAcertos = []

# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread1Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 60, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [4, 8]:
	
		# Testando resultados com diferentes quantidades de neuronios.
		for neuronios in [256, 512]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(neuronios) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,4):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential([
					 tf.keras.layers.Flatten(),
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
					])
					model.compile(optimizer=tf.train.GradientDescentOptimizer(0.17),
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_MLP1.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_MLP1.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(neuronios)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread2Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 60, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [4, 8]:
	
		# Testando resultados com diferentes quantidades de neuronios.
		for neuronios in [256, 512]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(neuronios) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,4):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential([
					 tf.keras.layers.Flatten(),
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
					])
					model.compile(optimizer=tf.train.GradientDescentOptimizer(0.17),
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_MLP2.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_MLP2.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(neuronios)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread3Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 60, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [4, 8]:
	
		# Testando resultados com diferentes quantidades de neuronios.
		for neuronios in [256, 512]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(neuronios) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,4):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential([
					 tf.keras.layers.Flatten(),
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
					])
					model.compile(optimizer=tf.train.GradientDescentOptimizer(0.17),
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_MLP3.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_MLP3.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(neuronios)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread4Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 60, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [4, 8]:
	
		# Testando resultados com diferentes quantidades de neuronios.
		for neuronios in [256, 512]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(neuronios) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,4):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential([
					 tf.keras.layers.Flatten(),
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(neuronios, activation=tf.nn.relu),
					 tf.keras.layers.Dropout(taxaDropout),# Diferentes valores de dropout.
					 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
					])
					model.compile(optimizer=tf.train.GradientDescentOptimizer(0.17),
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_MLP4.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_MLP4.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(neuronios)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
				
	
if __name__ == '__main__':

	
	camadas1 = threading.Thread(target=thread1Camadas,args=(1,))
	camadas2 = threading.Thread(target=thread2Camadas,args=(2,))
	camadas3 = threading.Thread(target=thread3Camadas,args=(3,))
	camadas4 = threading.Thread(target=thread4Camadas,args=(4,))
	
	camadas1.start()
	camadas2.start()
	camadas3.start()
	camadas4.start()
	
	try:
		camadas4.join(); 
	except:
		pass;
		
	try:
		camadas3.join(); 
	except:
		pass;
		
	try:
		camadas2.join(); 
	except:
		pass;
		
	try:
		camadas1.join(); 
	except:
		pass;
		
	listasFile = open("listas.txt", "w")
	listasFile.write(str(numeroDeNeuronios) + "\n")
	listasFile.write(str(numeroDeEpocas) + "\n")
	listasFile.write(str(numeroDeCamadas) + "\n")
	listasFile.write(str(numeroDeDropout) + "\n")
	listasFile.write(str(taxaDeAcertos) + "\n")
	listasFile.close()
		
		
		

In [ ]:
# q1Final.py

import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation=tf.nn.relu),
 tf.keras.layers.Dropout(0.2),
 tf.keras.layers.Dense(512, activation=tf.nn.relu),
 tf.keras.layers.Dropout(0.2),
 tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy'])
model.fit(x_train, y_train, epochs=8)
model.evaluate(x_test, y_test)
model_json = model.to_json()
json_file = open("model_MLP.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_MLP.h5")
print("Model saved to disk")
os.getcwd()

## Questão 2

Inicialmente, executou-se 5 vezes o código sugerido inicial a fim de verificar o desempenho da proposta. Seu desempenho foi de:

* Loss: 0.0260; Acurácia: 0.9909.

Utilizando-se o método de tentativa e erro, foi criado um script que verificava o desempenho da rede para diferentes parâmetros alterados como dropout (0.1 a 0.6), número de filtros (32 a 64), épocas de treinamento (2 a 6) e formato dos kernel utilizados (2x2 ou 3x3). O script executava esta mudança de parâmetros dentro de loops "for" para executar todas as combinações possíveis e tirava também a média das múltiplas execuções com mesmos parâmetros, a fim de se obter uma média de desempenho mais confiável. Os resultados desta varredura eram salvos ao final das execuções em um arquivo "listas.txt", permitindo ao usuários verificar qual a configuração obteve melhor desempenho.
Foram utilizadas 4 threads - para varredura de redes de 1 a 4 camadas - durante o treinamento, a fim de promover paralelismo e diminuir o tempo requerido, que era ainda maior que o demandado para a questão 1. Verificou-se que com 6 épocas de treinamento o resultdo era levemente melhorado, mas não siginficativamente. A variação das demais grandezas fazia o desempenho diminuir nos testes. Então, após a varredura, foi realizado mais uma tentativa de treinamento com adição de uma camada convolucional e dropout de 0.3, o que elevou os resultados e nos trouxe à proposta final de código para esta questão.

Para a proposta final deste modelo, os parâmetros alterados que resultaram no melhor desempenho durante a varredura foram:

* Adição de uma camada convolucional (seguida de uma max pool) após a saída da primeira layer de max pool; 6 épocas de treinamento. Os demais parâmetros foram mantidos por não apresentar vantagem média significativa.

O desempenho médio obtido foi de:

* Loss: 0.0190; Acurácia: 0.9931.

Ambas as soluções consumiram um tempo de execução da ordem de poucos minutos e a diferença de desempenho foi cerca de 0,22% em ganho.

Os arquivos utilizados foram (no diretório q2):

Proposta Inicial: q2Inicial.py
Script de Varredura de parâmetros: q2.py
Proposta Final: q2Final.py

## Comparação entre ELM, MLP e CNN

Desempenho:
* ELM: 91,09%
* MLP: 98,28%
* CNN: 99,31%

Nota-se claramente que a CNN apresenta o melhor desempenho dentre as 3 melhores técnicas utilizadas. Porém, o processo de treinamento para otimização desta toma dezenas de horas, enquanto que a ELM requer apenas alguns minutos para ser ajustada e ficar cerca de 8% abaixo em desempenho. Logo, se houver recursos computacionais, a CNN é a melhor escolha para este tipo de problema.

In [ ]:
# q2Inicial.py

import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# reshape to be [samples][width][height][pixels]
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
 activation='relu',
input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)
model_json = model.to_json()
json_file = open("model_CNN.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_CNN.h5")
print("Model saved to disk")
os.getcwd()

In [ ]:
# q2.py

import tensorflow as tf
import os
import threading

myMutex = threading.Lock()
value = "teste"

numeroDeNeuronios = []
numeroDeEpocas = []
numeroDeCamadas = []
numeroDeDropout = []
taxaDeAcertos = []

# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread1Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 40, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [2, 6]:
	
		# Testando resultados com diferentes quantidades de filtros.
		for filtros in [32, 64]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(filtros) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,3):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					# reshape to be [samples][width][height][pixels]
					x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
					x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential()
					model.add(tf.keras.layers.Conv2D(filtros, kernel_size=(3, 3),
					 activation='relu',
					input_shape=(28, 28, 1)))
					model.add(tf.keras.layers.Conv2D(filtros*2, (3, 3), activation='relu'))
					model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Flatten())
					model.add(tf.keras.layers.Dense(128, activation='relu'))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Dense(10, activation='softmax'))
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_CNN1.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_CNN1.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(filtros)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread2Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 40, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [2, 6]:
	
		# Testando resultados com diferentes quantidades de filtros.
		for filtros in [32, 64]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(filtros) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,3):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					# reshape to be [samples][width][height][pixels]
					x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
					x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential()
					model.add(tf.keras.layers.Conv2D(filtros, kernel_size=(2, 2),
					 activation='relu',
					input_shape=(28, 28, 1)))
					model.add(tf.keras.layers.Conv2D(filtros*2, (3, 3), activation='relu'))
					model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Flatten())
					model.add(tf.keras.layers.Dense(128, activation='relu'))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Dense(10, activation='softmax'))
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_CNN2.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_CNN2.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(filtros)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread3Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 40, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [2, 6]:
	
		# Testando resultados com diferentes quantidades de filtros.
		for filtros in [32, 64]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(filtros) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,3):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					# reshape to be [samples][width][height][pixels]
					x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
					x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential()
					model.add(tf.keras.layers.Conv2D(filtros, kernel_size=(3, 3),
					 activation='relu',
					input_shape=(28, 28, 1)))
					model.add(tf.keras.layers.Conv2D(filtros*2, (2, 2), activation='relu'))
					model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Flatten())
					model.add(tf.keras.layers.Dense(128, activation='relu'))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Dense(10, activation='softmax'))
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_CNN3.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_CNN3.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(filtros)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
# Vamos colocar uma thread para treinar cada rede com um numero especifico de camadas.
def thread4Camadas(camadas):
	
	# Para tirar a media das iteracoes, somaremos todas aqui e dividiremos pelo total.
	somaDasEficienciasDeCadaIteracao = 0
	
	# Os valores que utilizaremos para dropout variarao de 10% a 90% (instrucao abaixo).
	valoresDropout = range(10, 40, 10)# Variaremos de 10% em 10%.
	valoresDropout = [i/100 for i in valoresDropout]# Converte de porcentagem para escala de 0 a 1.
	
	# Testando resultados com diferentes quantidades de epocas.
	for epocas in [2, 6]:
	
		# Testando resultados com diferentes quantidades de filtros.
		for filtros in [32, 64]:
		
			# So para indicar em que passo da execucao estamos.
			print("\n\nepocas: " + str(epocas) + "\nCAMADAS" + str(camadas) + ": " + str(filtros) + "\n\n")
	
			# Este loop fica responsável por treinar com diferentes taxas de dropout.
			# "i" eh o valor a cada iteracao.
			for taxaDropout in valoresDropout:
			
				# Repetimos o treinamento algumas vezes para tirar uma media da eficiencia
				for iteracaoMedia in range(1,3):
					mnist = tf.keras.datasets.mnist
					(x_train, y_train),(x_test, y_test) = mnist.load_data()
					# reshape to be [samples][width][height][pixels]
					x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
					x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
					x_train, x_test = x_train / 255.0, x_test / 255.0
					model = tf.keras.models.Sequential()
					model.add(tf.keras.layers.Conv2D(filtros, kernel_size=(3, 3),
					 activation='relu',
					input_shape=(28, 28, 1)))
					model.add(tf.keras.layers.Conv2D(filtros*2, (3, 3), activation='relu'))
					model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Flatten())
					model.add(tf.keras.layers.Dense(128, activation='relu'))
					model.add(tf.keras.layers.Dropout(taxaDropout))
					model.add(tf.keras.layers.Dense(10, activation='softmax'))
					model.compile(optimizer='adam',
					 loss='sparse_categorical_crossentropy',
					 metrics=['accuracy'])
					model.fit(x_train, y_train, epochs=epocas)
					value = model.evaluate(x_test, y_test)
					model_json = model.to_json()
					json_file = open("model_CNN4.json", "w")
					json_file.write(model_json)
					json_file.close()
					model.save_weights("model_CNN4.h5")
					print("Model saved to disk")
					os.getcwd()
					
					somaDasEficienciasDeCadaIteracao = value[1] + somaDasEficienciasDeCadaIteracao
					
				
				
				
				myMutex.acquire()
				numeroDeNeuronios.append(filtros)
				numeroDeEpocas.append(epocas)
				numeroDeCamadas.append(camadas)
				numeroDeDropout.append(taxaDropout)
				taxaDeAcertos.append(somaDasEficienciasDeCadaIteracao/iteracaoMedia)
				myMutex.release()
				
				# Reiniciamos a soma.
				somaDasEficienciasDeCadaIteracao = 0
				
				
	
if __name__ == '__main__':

	
	camadas1 = threading.Thread(target=thread1Camadas,args=(1,))
	camadas2 = threading.Thread(target=thread2Camadas,args=(2,))
	camadas3 = threading.Thread(target=thread3Camadas,args=(3,))
	camadas4 = threading.Thread(target=thread4Camadas,args=(4,))
	
	camadas1.start()
	camadas2.start()
	camadas3.start()
	camadas4.start()
	
	try:
		camadas4.join(); 
	except:
		pass;
		
	try:
		camadas3.join(); 
	except:
		pass;
		
	try:
		camadas2.join(); 
	except:
		pass;
		
	try:
		camadas1.join(); 
	except:
		pass;
		
	listasFile = open("listas.txt", "w")
	listasFile.write(str(numeroDeNeuronios) + "\n")
	listasFile.write(str(numeroDeEpocas) + "\n")
	listasFile.write(str(numeroDeCamadas) + "\n")
	listasFile.write(str(numeroDeDropout) + "\n")
	listasFile.write(str(taxaDeAcertos) + "\n")
	listasFile.close()
		
		
		

In [ ]:
# q2Final.py
import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# reshape to be [samples][width][height][pixels]
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
 activation='relu',
input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy'])
model.fit(x_train, y_train, epochs=6)
model.evaluate(x_test, y_test)
model_json = model.to_json()
json_file = open("model_CNN.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_CNN.h5")
print("Model saved to disk")
os.getcwd()

## Questão 3

A substituição do bloco RNN pelo GRU que estava comentado levou a uma pequena melhoria de desempenho. A alteração de diretório para o arquivo CSV foi feita de forma que basta eles estarem no mesmo diretório para o programa funcionar.

(Tentei colocar as imagens abaixo, mas consegui apenas colocá-las no diretório q3 da questão 3).



In [ ]:
import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf

# split data in 80%/10%/10% train/validation/test sets
valid_set_size_percentage = 10 
test_set_size_percentage = 10 

#display parent directory and working directory
print(os.path.dirname(os.getcwd())+':', os.listdir(os.path.dirname(os.getcwd())));
print(os.getcwd()+':', os.listdir(os.getcwd()));



# import all stock prices 
df = pd.read_csv("prices-split-adjusted.csv", index_col = 0)
df.info()
df.head()

# number of different stocks
print('\nnumber of different stocks: ', len(list(set(df.symbol))))
print(list(set(df.symbol))[:10])


df.tail()


df.describe()


df.info()


plt.figure(figsize=(15, 5));
plt.subplot(1,2,1);
plt.plot(df[df.symbol == 'EQIX'].open.values, color='red', label='open')
plt.plot(df[df.symbol == 'EQIX'].close.values, color='green', label='close')
plt.plot(df[df.symbol == 'EQIX'].low.values, color='blue', label='low')
plt.plot(df[df.symbol == 'EQIX'].high.values, color='black', label='high')
plt.title('stock price')
plt.xlabel('time [days]')
plt.ylabel('price')
plt.legend(loc='best')
#plt.show()

plt.subplot(1,2,2);
plt.plot(df[df.symbol == 'EQIX'].volume.values, color='black', label='volume')
plt.title('stock volume')
plt.xlabel('time [days]')
plt.ylabel('volume')
plt.legend(loc='best');


# function for min-max normalization of stock
def normalize_data(df):
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    df['open'] = min_max_scaler.fit_transform(df.open.values.reshape(-1,1))
    df['high'] = min_max_scaler.fit_transform(df.high.values.reshape(-1,1))
    df['low'] = min_max_scaler.fit_transform(df.low.values.reshape(-1,1))
    df['close'] = min_max_scaler.fit_transform(df['close'].values.reshape(-1,1))
    return df

# function to create train, validation, test data given stock data and sequence length
def load_data(stock, seq_len):
    data_raw = stock.as_matrix() # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len): 
        data.append(data_raw[index: index + seq_len])
    
    data = np.array(data);
    valid_set_size = int(np.round(valid_set_size_percentage/100*data.shape[0]));  
    test_set_size = int(np.round(test_set_size_percentage/100*data.shape[0]));
    train_set_size = data.shape[0] - (valid_set_size + test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_valid = data[train_set_size:train_set_size+valid_set_size,:-1,:]
    y_valid = data[train_set_size:train_set_size+valid_set_size,-1,:]
    
    x_test = data[train_set_size+valid_set_size:,:-1,:]
    y_test = data[train_set_size+valid_set_size:,-1,:]
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

# choose one stock
df_stock = df[df.symbol == 'EQIX'].copy()
df_stock.drop(['symbol'],1,inplace=True)
df_stock.drop(['volume'],1,inplace=True)

cols = list(df_stock.columns.values)
print('df_stock.columns.values = ', cols)

# normalize stock
df_stock_norm = df_stock.copy()
df_stock_norm = normalize_data(df_stock_norm)

# create train, test data
seq_len = 20 # choose sequence length
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(df_stock_norm, seq_len)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)



plt.figure(figsize=(15, 5));
plt.plot(df_stock_norm.open.values, color='red', label='open')
plt.plot(df_stock_norm.close.values, color='green', label='low')
plt.plot(df_stock_norm.low.values, color='blue', label='low')
plt.plot(df_stock_norm.high.values, color='black', label='high')
#plt.plot(df_stock_norm.volume.values, color='gray', label='volume')
plt.title('stock')
plt.xlabel('time [days]')
plt.ylabel('normalized price/volume')
plt.legend(loc='best')
plt.show()



## Basic Cell RNN in tensorflow

index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

# parameters
n_steps = seq_len-1 
n_inputs = 4 
n_neurons = 200 
n_outputs = 4
n_layers = 2
learning_rate = 0.001
batch_size = 50
n_epochs = 100 
train_set_size = x_train.shape[0]
test_set_size = x_test.shape[0]

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

## use Basic RNN Cell
#layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.elu)
#          for layer in range(n_layers)]

# use Basic LSTM Cell 
#layers = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.elu)
#          for layer in range(n_layers)]

# use LSTM Cell with peephole connections
#layers = [tf.contrib.rnn.LSTMCell(num_units=n_neurons, 
#                                  activation=tf.nn.leaky_relu, use_peepholes = True)
#          for layer in range(n_layers)]

# use GRU cell
layers = [tf.contrib.rnn.GRUCell(num_units=n_neurons, activation=tf.nn.leaky_relu)
          for layer in range(n_layers)]
                                                                     
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons]) 
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
outputs = outputs[:,n_steps-1,:] # keep only last output of sequence
                                              
loss = tf.reduce_mean(tf.square(outputs - y)) # loss function = mean squared error 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
training_op = optimizer.minimize(loss)
                                              
# run graph
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    for iteration in range(int(n_epochs*train_set_size/batch_size)):
        x_batch, y_batch = get_next_batch(batch_size) # fetch the next training batch 
        sess.run(training_op, feed_dict={X: x_batch, y: y_batch}) 
        if iteration % int(5*train_set_size/batch_size) == 0:
            mse_train = loss.eval(feed_dict={X: x_train, y: y_train}) 
            mse_valid = loss.eval(feed_dict={X: x_valid, y: y_valid}) 
            print('%.2f epochs: MSE train/valid = %.6f/%.6f'%(
                iteration*batch_size/train_set_size, mse_train, mse_valid))

    y_train_pred = sess.run(outputs, feed_dict={X: x_train})
    y_valid_pred = sess.run(outputs, feed_dict={X: x_valid})
    y_test_pred = sess.run(outputs, feed_dict={X: x_test})
    
    
    
    
y_train.shape



ft = 0 # 0 = open, 1 = close, 2 = highest, 3 = lowest

## show predictions
plt.figure(figsize=(15, 5));
plt.subplot(1,2,1);

plt.plot(np.arange(y_train.shape[0]), y_train[:,ft], color='blue', label='train target')

plt.plot(np.arange(y_train.shape[0], y_train.shape[0]+y_valid.shape[0]), y_valid[:,ft],
         color='gray', label='valid target')

plt.plot(np.arange(y_train.shape[0]+y_valid.shape[0],
                   y_train.shape[0]+y_test.shape[0]+y_test.shape[0]),
         y_test[:,ft], color='black', label='test target')

plt.plot(np.arange(y_train_pred.shape[0]),y_train_pred[:,ft], color='red',
         label='train prediction')

plt.plot(np.arange(y_train_pred.shape[0], y_train_pred.shape[0]+y_valid_pred.shape[0]),
         y_valid_pred[:,ft], color='orange', label='valid prediction')

plt.plot(np.arange(y_train_pred.shape[0]+y_valid_pred.shape[0],
                   y_train_pred.shape[0]+y_valid_pred.shape[0]+y_test_pred.shape[0]),
         y_test_pred[:,ft], color='green', label='test prediction')

plt.title('past and future stock prices')
plt.xlabel('time [days]')
plt.ylabel('normalized price')
plt.legend(loc='best');

plt.subplot(1,2,2);

plt.plot(np.arange(y_train.shape[0], y_train.shape[0]+y_test.shape[0]),
         y_test[:,ft], color='black', label='test target')

plt.plot(np.arange(y_train_pred.shape[0], y_train_pred.shape[0]+y_test_pred.shape[0]),
         y_test_pred[:,ft], color='green', label='test prediction')

plt.title('future stock prices')
plt.xlabel('time [days]')
plt.ylabel('normalized price')
plt.legend(loc='best');

corr_price_development_train = np.sum(np.equal(np.sign(y_train[:,1]-y_train[:,0]),
            np.sign(y_train_pred[:,1]-y_train_pred[:,0])).astype(int)) / y_train.shape[0]
corr_price_development_valid = np.sum(np.equal(np.sign(y_valid[:,1]-y_valid[:,0]),
            np.sign(y_valid_pred[:,1]-y_valid_pred[:,0])).astype(int)) / y_valid.shape[0]
corr_price_development_test = np.sum(np.equal(np.sign(y_test[:,1]-y_test[:,0]),
            np.sign(y_test_pred[:,1]-y_test_pred[:,0])).astype(int)) / y_test.shape[0]

print('correct sign prediction for close - open price for train/valid/test: %.2f/%.2f/%.2f'%(corr_price_development_train, corr_price_development_valid, corr_price_development_test))